In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
from tqdm.notebook import tqdm
import IPython

In [13]:
#Katie
games = pd.read_csv("games.csv")
plays = pd.read_csv("plays.csv")
players = pd.read_csv("players.csv")
week = pd.read_csv("tracking_week_1.csv")
tackles = pd.read_csv("tackles.csv")


week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

(1407439, 18)

In [ ]:
#Emily 
data_dir = os.getcwd() + '/nfl-big-data-bowl-2024/'
players = pd.read_csv(data_dir + "players.csv")
week = pd.read_csv(data_dir + "tracking_week_1.csv")
plays = pd.read_csv(data_dir + "plays.csv")
tackles = pd.read_csv(data_dir + "tackles.csv")


# join player positioning information onto a week's worth of tracking data 
week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

In [ ]:
#Matts reading in files 
def load_dataset(dataset_name):
    """
    Download a specific dataset from data directory.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    return pd.read_csv(f"C:\\Users\\mattd\\Documents\\GitHub\\big-data-bowl-2024\\data\\{dataset_name}.csv")

# Read In csvs
plays = load_dataset("plays")
players = load_dataset("players")
week = load_dataset("tracking_week_1")
tackles = load_dataset("tackles")
week = week.merge(players.loc[:, ['nflId', 'position']], how='left')
week.shape

In [14]:
"""Game level functions"""

def get_frames_of_catch(games):
    """
    Returns only the frame of the play where the ball is caught.

    Parameters:
    - games: Dataset of games

    Returns:
    - frame: The tracking data of only the data where the ball is caught
    """
    return games.loc[games['event'] == 'pass_outcome_caught'].copy()

In [15]:
"""Frame level functions"""
def distance_to_ball(play_data, frame, ball_carrier_id):
    """
    Calculates the distance to the ball for each player per frame in a play.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    ball_carrier = frame[frame["nflId"] == ball_carrier_id]
    carr_x = ball_carrier["x"].values[0]
    carr_y = ball_carrier["y"].values[0]
    frame['distance_to_ball_carrier'] = cdist(frame[['x', 'y']], [[carr_x, carr_y]], metric='euclidean')
    frame['ballCarrierId'] = ball_carrier_id  # Include ballCarrierId in the frame
    return frame

def who_tackles(play_data, frame):
    """
    Adds who tackles the ball carrier to the tracking data set.

    Parameters:
    - dataset_name: Name of the dataset to load
    """
    ball_carrier_id = play_data["ballCarrierId"].iloc[0]
    ball_carrier = frame[frame["nflId"] == ball_carrier_id]
    carr_x = ball_carrier["x"].values[0]
    carr_y = ball_carrier["y"].values[0]
    frame['distance_to_ball_carrier'] = cdist(frame[['x', 'y']], [[carr_x, carr_y]], metric='euclidean')
    return frame

def create_dnn_input_df(frames, side_of_ball):
    """
    Creates the input dataframe for the DNN model.

    Parameters:
    - frames: DataFrame containing player tracking data
    - side_of_ball: String representing the side of the ball (e.g., 'offense', 'defense')
    """
    # Sort the players first by position, then by jersey number

    sorted = frames.sort_values(by=['position', 'jerseyNumber']).reset_index(drop=True)

    input_df = pd.DataFrame()
    
    # Iterate over each row in the sorted DataFrame
    for idx, row in sorted.iterrows():
        # Extract player's x and y values
        x = row['x']
        y = row['y']
        s = row['s']
        a = row['a']
        dis = row['dis']
        o = row['o']
        dir = row['dir']
        distance_to_ball_carrier = row['distance_to_ball_carrier']

        # Add columns to the new DataFrame
        input_df.loc[0,f'{side_of_ball}_x_{idx}'] = x
        input_df.loc[0,f'{side_of_ball}_y_{idx}'] = y
        input_df.loc[0,f'{side_of_ball}_s_{idx}'] = s
        input_df.loc[0,f'{side_of_ball}_a_{idx}'] = a
        input_df.loc[0,f'{side_of_ball}_dis_{idx}'] = dis
        input_df.loc[0,f'{side_of_ball}_o_{idx}'] = o
        input_df.loc[0,f'{side_of_ball}_dir_{idx}'] = dir
        input_df.loc[0,f'{side_of_ball}_distance_to_ball_carrier_{idx}'] = distance_to_ball_carrier

    return input_df


def calculate_nearby_players(play, radius=5):
    
    """
    Calculate the number of defenders and blockers within a specified radius of the ball carrier.
    
    Parameters:
    play - DataFrame containing play data including player positions and teams
    radius - Distance within which a player is considered 'nearby' ( 5 yards)
    
    Returns:
    DataFrame with additional columns 'defenders_nearby' and 'blockers_nearby'
    """
    
    ball_carrier_id = play["ballCarrierId"].iloc[0]
    ball_carrier = play[play["nflId"] == ball_carrier_id]

    # Calculate distances from the ball carrier to all players
    play['distance_to_carrier'] = np.sqrt((play['x'] - ball_carrier['x'].values[0])**2 + (play['y'] - ball_carrier['y'].values[0])**2)

    # Identify defenders and blockers
    is_defender = (play['club'] != ball_carrier['club'].values[0]) & (play['nflId'] != ball_carrier_id)
    is_blocker = (play['club'] == ball_carrier['club'].values[0]) & (play['nflId'] != ball_carrier_id)

    # Count nearby defenders and blockers
    defenders_nearby = play[is_defender & (play['distance_to_carrier'] < radius)].groupby(['gameId', 'playId']).nflId.nunique().rename('defenders_nearby')
    blockers_nearby = play[is_blocker & (play['distance_to_carrier'] < radius)].groupby(['gameId', 'playId']).nflId.nunique().rename('blockers_nearby')

    return defenders_nearby, blockers_nearby

In [16]:
def calculate_snap_to_catch_time(plays, tracking):
    """
    Calculates the time in milliseconds from ball_snap to pass_outcome_caught for each play.

    Parameters:
    - plays: DataFrame containing play data
    - tracking: DataFrame containing tracking data

    Returns:
    - DataFrame with gameId, playId, and time difference in milliseconds
    """
    time_diffs = []

    for pid in plays['playId'].unique():
        # Get the gameId for the current playId from the plays DataFrame
        gid = plays[plays['playId'] == pid]['gameId'].iloc[0]
        
        play_tracking = tracking[tracking['playId'] == pid]

        snap_tracking = play_tracking[play_tracking['event'] == 'ball_snap']
        catch_tracking = play_tracking[play_tracking['event'] == 'pass_outcome_caught']

        if not snap_tracking.empty and not catch_tracking.empty:
            snap_time = pd.to_datetime(snap_tracking['time'].iloc[0])
            catch_time = pd.to_datetime(catch_tracking['time'].iloc[0])

            time_diff = (catch_time - snap_time).total_seconds() * 1000
            time_diffs.append({'gameId': gid, 'playId': pid, 'time_diff_ms': time_diff})
        else:
            time_diffs.append({'gameId': gid, 'playId': pid, 'time_diff_ms': None})

    return pd.DataFrame(time_diffs)


def merge_play_data(final_df, plays_df, columns_to_merge):
    """
    Merges specified columns from the plays DataFrame into the final DataFrame.

    Parameters:
    - final_df: The final DataFrame to which the columns will be added.
    - plays_df: The DataFrame containing play data.
    - columns_to_merge: List of column names to be merged from plays_df to final_df.

    Returns:
    - DataFrame with merged columns.
    """
    # Select required columns from plays DataFrame along with gameId and playId
    plays_subset = plays_df[['gameId', 'playId'] + columns_to_merge]

    # Merge the plays data with the final DataFrame
    merged_df = final_df.merge(plays_subset, on=['gameId', 'playId'], how='left')
    return merged_df

# Usage of the function
columns_to_add = ['quarter', 'down', 'yardsToGo', 'gameClock', 'passLength', 'expectedPoints']


In [19]:
# Process each game, play, and frame
pass_only = get_frames_of_catch(week)
#time_diffs_df = calculate_snap_to_catch_time(plays, week)-- this was for the calculate_snap_to_catch_time


final_input_list = []
for gid in tqdm(pass_only['gameId'].unique(), leave=True):
    
    # print(f"Processing Game ID: {gid}")  # Printing Game ID 
    game = pass_only.loc[pass_only['gameId'] == gid].copy()
    game_plays = plays[plays['gameId'] == gid]
    game_tackles = tackles[tackles['gameId'] == gid]

    for pid in tqdm(game['playId'].unique(), leave=False):
        #print(f"  Processing Play ID: {pid}")  # Print Play ID
        play = game.loc[game['playId'] == pid].copy()
        play_data = game_plays[game_plays['playId'] == pid]
        ball_carrier_id = play_data["ballCarrierId"].iloc[0]  # Extract ballCarrierId

        
        #print("Columns in play_data:", play_data.columns)


        #Adding distance to ball carrier as a feature in tracking data
        play = distance_to_ball(play_data, play, ball_carrier_id)  # Pass ballCarrierId

        
        #print("Columns in play after distance_to_ball:", play.columns)

            
        defenders_nearby, blockers_nearby = calculate_nearby_players(play)

        offense = play[play["club"] == play_data["possessionTeam"].iloc[0]]
        defense = play[play["club"] == play_data["defensiveTeam"].iloc[0]]
        #print(defense.head())
        #print("Length of ",len(defense))
        #print("Length of ",len(offense))


        offense_input = create_dnn_input_df(offense, "o")
        defense_input = create_dnn_input_df(defense, "d")
        
        # Including the game and play IDs in the offense and defense dataframes
        offense_input['gameId'] = gid
        defense_input['gameId'] = gid
        offense_input['playId'] = pid
        defense_input['playId'] = pid


        
        player_tracking_data = pd.concat([offense_input, defense_input], axis=1)
        player_tracking_data['defenders_nearby'] = defenders_nearby.get((gid, pid), 0)
        player_tracking_data['blockers_nearby'] = blockers_nearby.get((gid, pid), 0)
        
        
        final_input_list.append(player_tracking_data)

        
final_input_df = pd.concat(final_input_list, ignore_index=True)

#this was for the calculate_snap_to_catch_time
#final_input_df = final_input_df.merge(time_diffs_df, on=['gameId', 'playId'], how='left')


In [20]:
final_input_df

,o_x_0,o_y_0,o_s_0,o_a_0,o_dis_0,o_o_0,o_dir_0,o_distance_to_ball_carrier_0,o_x_1,o_y_1,...,d_s_10,d_a_10,d_dis_10,d_o_10,d_dir_10,d_distance_to_ball_carrier_10,gameId,playId,defenders_nearby,blockers_nearby
0,88.21,29.31,1.42,0.64,0.14,282.32,347.15,10.456003,91.02,31.34,...,1.79,4.33,0.19,82.14,305.29,21.893600,2022090800,56,2,0
1,72.81,31.22,2.06,1.26,0.21,8.37,79.87,8.712204,70.66,32.44,...,5.02,3.68,0.51,5.20,310.45,30.639067,2022090800,122,2,0
2,61.82,24.53,1.18,2.14,0.12,259.03,110.68,10.675917,60.26,25.86,...,2.97,4.35,0.27,114.51,144.12,9.193808,2022090800,167,1,0
3,42.62,30.35,2.34,1.23,0.24,316.92,325.71,17.242796,45.24,35.12,...,6.18,2.36,0.62,10.45,4.93,19.972351,2022090800,212,1,0
4,38.13,38.47,1.87,0.82,0.20,326.44,326.44,19.881481,36.38,25.86,...,7.56,0.96,0.76,299.98,313.74,8.202103,2022090800,236,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,87.61,25.38,2.79,1.02,0.29,314.18,356.02,15.582997,90.91,21.13,...,2.28,1.87,0.24,305.64,257.22,22.875861,2022091200,3596,1,0
686,81.38,32.02,0.94,0.53,0.10,316.81,38.20,12.777437,82.61,32.43,...,3.73,3.86,0.35,303.91,293.36,17.898338,2022091200,3628,2,0
687,74.56,30.32,1.68,1.56,0.17,306.78,333.18,10.440278,77.13,32.79,...,2.37,2.26,0.24,281.55,345.66,9.617614,2022091200,3723,1,0
688,62.84,33.28,3.91,1.29,0.39,314.68,341.62,3.023640,62.88,35.32,...,0.91,2.78,0.10,234.01,151.04,9.308861,2022091200,3747,2,2
